<a href="https://colab.research.google.com/github/VictorCastroDeFaria/LinearProgramming/blob/main/ProblemasLista4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install pulp

import pulp
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Implementação do problema


cuts_and_types = [[5, 2, 1, 0, 0, 0, 0, 0, 0, 1],
                  [0, 0, 1, 2, 0, 0, 3, 1, 0, 0],
                  [0, 0, 0, 2, 1, 0, 0, 0, 1, 0],
                  [0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
                  [0, 0, 1, 0, 0, 1, 0, 0, 0, 0]]

coils_demand = [18, 31, 25, 15, 14]

cuts_amount = 10 # tipos de padrões de corte

P = pulp.LpVariable.dicts('CuttingPattern', range(cuts_amount), lowBound = 0, cat = 'Integer')

In [7]:
# Verificação de viabilidade do problema

problem = LpProblem("Problema da minimização de custos", LpMinimize)

problem += lpSum([P[j] for j in range(cuts_amount)])

coils_amount = 5 # Tipos de bobinas

for i in range(coils_amount):
  problem += lpSum([P[j]*cuts_and_types[i][j] for j in range(cuts_amount)]) == coils_demand[i]

problem.solve()

print("Status:", LpStatus[problem.status])

print("objective = %s" % value(problem.objective))

for j in range(cuts_amount):
  print(int(P[j].varValue),end = ' ')

Status: Optimal
objective = 37.0
0 0 14 7 11 0 1 0 0 4 

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [8]:
# Adaptação do problema com incorporação de custos

Pb = pulp.LpVariable.dicts('CuttingPatternUsage', range(cuts_amount), lowBound = 0, cat = 'Binary')
big_M = 10000

problem2 = LpProblem("Problema da minimização de custos", LpMinimize)

problem2 += lpSum([0.25*P[j] for j in range(cuts_amount)]) + 3*lpSum([Pb[j] for j in range(cuts_amount)])

coils_amount = 5 # Tipos de bobinas

for i in range(coils_amount):
  problem2 += lpSum([P[j]*cuts_and_types[i][j] for j in range(cuts_amount)]) == coils_demand[i]

for j in range(cuts_amount):
  problem2 += P[j] <= big_M*Pb[j]

problem2.solve()

print("Status:", LpStatus[problem2.status])

print("objective = %s" % value(problem2.objective))

for j in range(cuts_amount):
  print(int(P[j].varValue),end = ' ')
print(' ')
for j in range(cuts_amount):
  print(int(Pb[j].varValue),end = ' ')

Status: Optimal
objective = 24.25
0 0 14 7 11 0 1 0 0 4  
0 0 1 1 1 0 1 0 0 1 

In [9]:
# Adaptação do problema para a possibilidade de estoque, com incorporação de custos

Pb = pulp.LpVariable.dicts('CuttingPatternUsage', range(cuts_amount), lowBound = 0, cat = 'Binary')
big_M = 10000

problem3 = LpProblem("Problema da minimização de custos", LpMinimize)

problem3 += lpSum([0.25*P[j] for j in range(cuts_amount)]) + 3*lpSum([Pb[j] for j in range(cuts_amount)])

coils_amount = 5 # Tipos de bobinas

for i in range(coils_amount):
  problem3 += lpSum([P[j]*cuts_and_types[i][j] for j in range(cuts_amount)]) >= coils_demand[i]

for j in range(cuts_amount):
  problem3 += P[j] <= big_M*Pb[j]

problem3.solve()

print("Status:", LpStatus[problem3.status])

print("objective = %s" % value(problem3.objective))

for j in range(cuts_amount):
  print(int(P[j].varValue),end = ' ')
print(' ')
for j in range(cuts_amount):
  print(int(Pb[j].varValue),end = ' ')

Status: Optimal
objective = 19.0
0 0 18 7 15 0 0 0 0 0  
0 0 1 1 1 0 0 0 0 0 